	Refuerzo de conceptos: 
		- Al volver a trabajar en el proyecto, consolidarás lo que has aprendido y entenderás mejor cómo aplicar diferentes técnicas y herramientas.
	Perfeccionamiento del código: 
		-Puedes refactorizar y optimizar tu código, mejorando la eficiencia y la claridad. Es una buena oportunidad para aplicar mejores prácticas que quizás no pudiste incluir antes.
	Exploración de nuevas ideas: 
		- Con más tiempo, puedes experimentar con diferentes enfoques o añadir características que no habías considerado antes. Por ejemplo, probar distintos algoritmos de machine learning, usar técnicas de visualización más avanzadas, o incluso implementar un pipeline de datos más robusto.
	Portafolio: 
		- Un proyecto más completo y bien elaborado será una excelente adición a tu portafolio, lo que te será útil para futuras oportunidades laborales o académicas.

## 00.01 Importaciones

In [1]:
# Manejo de archivos y tiempo
import os
import json
from time import sleep
from datetime import datetime, timedelta

# NumPy y Pandas
import numpy as np
import pandas as pd

# Visualización
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Escalado
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# GridSearchCV y Validación
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold, StratifiedKFold

# Métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report

# Modelos clásicos de series temporales
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Modelos de machine learning
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# PyTorch (Alternativa a Keras/TensorFlow)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Guardado y carga de modelos
import joblib

# Manejo de advertencias
import warnings

In [2]:
# Quiero ver todas las columnas sin elípsis.

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## 00.02 Declaración de funciones

In [3]:
# def llamar_archivos(ruta_archivo):
#      lista_csv = []

#      for csv in os.listdir(ruta_archivo):
#          if csv.endswith('.csv'):
#             path_csv = os.path.join(ruta_archivo, csv)
#             df = pd.read_csv(path_csv)
#             lista_csv.append(df)

#      return pd.concat(lista_csv, ignore_index = True)



In [4]:
def llamar_archivos(ruta_archivo, dato = ''):
    lista = []

    for archivo in os.listdir(ruta_archivo):
        path_archivo = os.path.join(ruta_archivo, archivo)
        df = pd.DataFrame()
        if archivo.startswith(dato) & archivo.endswith('.csv'):
            df = pd.read_csv(path_archivo)
        elif archivo.startswith(dato) & archivo.endswith('.json'):
            df = pd.read_json(path_archivo)
        lista.append(df)
            
    return pd.concat(lista, ignore_index = True)

## 01 EDA

### 01.01 EDA Fitbit

#### 01.01.01 Account Changes

- El archivo CSV de Account_Management_Events en los datos exportados de Fitbit generalmente contiene información relacionada con eventos de gestión de la cuenta. Estos eventos suelen registrar actividades como la creación de la cuenta, inicios de sesión, cambios de configuración, sincronización de dispositivos, y otras acciones administrativas realizadas en mi cuenta de Fitbit.

Columnas que se hallan en este archivo CSV y tipo de datos que proporcionan:

- **Timestamp:** 
    - Este campo indica la fecha y hora en que ocurrió el evento. Es útil para rastrear cuándo se realizaron ciertos cambios o actividades en tu cuenta.
- **Event Type:** 
    - Describe el tipo de evento que se registró. Esto podría incluir eventos como “Login”, “Account Created”, “Device Sync”, “Settings Changed”, entre otros.
- **Device:** 
    - Información sobre el dispositivo asociado con el evento, si corresponde. Esto podría incluir el nombre o tipo de dispositivo Fitbit que usaste.
- **User ID:** 
    - Una identificación única asociada a tu cuenta de usuario. Este campo es importante para distinguir eventos en cuentas con múltiples usuarios o dispositivos.
- **IP Address:** 
    - La dirección IP desde la cual se realizó el evento, si está disponible. Esto puede ser útil para identificar desde qué ubicación se accedió a tu cuenta.
- **Location:** 
    - Información geográfica basada en la IP, como país o ciudad, asociada con el evento. Esto te da una idea de dónde ocurrió la actividad.
- **Device OS**: 
    - El sistema operativo del dispositivo utilizado durante el evento (por ejemplo, iOS, Android).
- **Browser:** 
    - Si el evento ocurrió en un navegador web, este campo puede registrar el nombre del navegador (Chrome, Firefox, etc.).
- **App Version:** 
    - Si el evento ocurrió en la aplicación móvil, este campo podría indicar la versión de la aplicación en uso.

In [5]:
path_account_m_e = 'Datos/Fitbit/Account Changes'

account_df = llamar_archivos(path_account_m_e)
account_df.head(4)

,timestamp,event_name,email,location,ip,outcome,reason
0,Wed Oct 11 19:10:57 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
1,Wed Oct 11 18:14:20 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
2,Wed Oct 11 18:13:49 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
3,Wed Oct 11 15:47:00 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",84.125.86.208,SUCCESS,NaN


In [6]:
col_account = {
    'timestamp': 'Fecha y Hora',
    'event_name': 'Evento',
    'email': 'Correo Electrónico',
    'location': 'Ubicación',
    'ip': 'Direccion IP',
    'outcome': 'Resultado',
    'reason': 'Motivo'
}

account_df = account_df.rename(columns = col_account)
account_df.head(4)

,Fecha y Hora,Evento,Correo Electrónico,Ubicación,Direccion IP,Resultado,Motivo
0,Wed Oct 11 19:10:57 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
1,Wed Oct 11 18:14:20 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
2,Wed Oct 11 18:13:49 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
3,Wed Oct 11 15:47:00 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",84.125.86.208,SUCCESS,NaN


#### 01.01.02 Active Zone Minutes (AZM)

- Los “Active Zone Minutes” (minutos en zona activa) son un tipo de dato que Fitbit proporciona para ayudar a entender mejor el nivel de actividad física y su impacto en la salud. Este dato es importante porque refleja el tiempo que se pasa en diferentes zonas de frecuencia cardíaca durante actividades físicas, lo que permite evaluar la intensidad de los ejercicios y su beneficio para la salud cardiovascular.

Columnas que se hallan en este archivo CSV y tipo de datos que proporcionan:

- **Timestamp (Fecha y Hora):**
    -  Este campo indica el momento exacto en que se registraron los minutos en zona activa. Es útil para ver cuándo ocurrieron los períodos de mayor actividad.
- **Zona de Frecuencia Cardíaca:**
	- **Fat Burn Zone (Zona de quema de grasa):**
		- Refleja la cantidad de minutos en los que tu frecuencia cardíaca estuvo en una zona moderada, generalmente entre el 50% y el 70% de tu frecuencia cardíaca máxima. En esta zona, tu cuerpo está utilizando principalmente grasa como fuente de energía.
		- Cardio Zone (Zona de cardio): Representa los minutos en los que tu frecuencia cardíaca estuvo entre el 70% y el 85% de tu frecuencia cardíaca máxima. Este nivel de esfuerzo mejora la capacidad cardiovascular y la eficiencia del corazón.
		- Peak Zone (Zona de pico): Indica los minutos en los que tu frecuencia cardíaca estuvo por encima del 85% de tu frecuencia cardíaca máxima, generalmente durante ejercicios de alta intensidad. Esta zona es efectiva para mejorar la capacidad cardiovascular y quemar calorías rápidamente.
- **Total Active Zone Minutes (Total de minutos en zona activa):** 
	- Este campo refleja la suma de los minutos que pasaste en las distintas zonas de frecuencia cardíaca. Es un indicador global de tu nivel de actividad física intensa y moderada.
- **Duración del Ejercicio:** 
	- Algunos registros también pueden incluir la duración total del ejercicio o actividad física que contribuyó a esos minutos en zona activa.

¿Por qué son importantes estos datos?

Los minutos en zona activa permiten evaluar la intensidad de los ejercicios, algo que solo contar pasos no podría mostrar. Cuanto más tiempo se pase en zonas de mayor frecuencia cardíaca, más beneficios cardiovasculares se obtienen. Fitbit establece objetivos de minutos en zona activa semanales recomendados por organizaciones de salud como la Organización Mundial de la Salud (OMS). Alcanzar estos objetivos está relacionado con la mejora de la salud cardiovascular y la reducción del riesgo de enfermedades crónicas. Además, se pueden usar estos datos para analizar la efectividad de los entrenamientos. Por ejemplo, si se ve que se pasa poco tiempo en zonas de mayor intensidad, se podría ajustar las rutinas para obtener mejores resultados.

In [7]:
path_active_z_m = 'Datos/Fitbit/Active Zone Minutes (AZM)'

active_z_m = llamar_archivos(path_active_z_m)
active_z_m.head(4)

,date_time,heart_zone_id,total_minutes
0,2022-12-01T14:52,FAT_BURN,1
1,2022-12-02T11:59,FAT_BURN,1
2,2022-12-02T12:04,CARDIO,2
3,2022-12-02T14:27,FAT_BURN,1


#### 01.01.03 Activity goals

Tipos de Datos que Puedes Encontrar en “Activity Goals”:
- **Daily Step Goal (Objetivo diario de pasos):** 
	- El número de pasos que te has propuesto caminar cada día. Es uno de los objetivos más comunes que los usuarios de Fitbit establecen para mantenerse activos.
- **Daily Distance Goal (Objetivo diario de distancia):**
	- La distancia que te propones recorrer cada día, medida en kilómetros o millas, dependiendo de tu configuración.
- **Daily Calorie Burn Goal (Objetivo diario de calorías quemadas):**
	- La cantidad de calorías que te has propuesto quemar cada día a través de la actividad física.
- **Daily Active Minutes Goal (Objetivo diario de minutos activos):**
	-  El tiempo que te propones estar activo cada día. Esto se mide en minutos en los que estás realizando actividad física moderada a intensa.
- **Weekly Exercise Goal (Objetivo semanal de ejercicio):** 
	- Una meta que puedes establecer para la cantidad de días por semana en los que planeas realizar ejercicio. Por ejemplo, podrías establecer un objetivo de ejercitarte al menos 5 días a la semana.
- **Daily Floors Climbed Goal (Objetivo diario de pisos subidos):**
	- El número de pisos (tramos de escaleras) que te propones subir cada día.


    

In [8]:
path_activity_goals = 'Datos/Fitbit/Activity Goals'

activity_goals = llamar_archivos(path_activity_goals)
activity_goals.head(4)

,type,frequency,target,result,status,is_primary,start_date,end_date,created_on,edited_on
0,ACTIVE_ZONE_MINUTES_GOAL,DAILY,30.0,NaN,NEW,False,NaN,NaN,2024-04-24T08:00:33.959+00:00,NaN
1,ACTIVE_ZONE_MINUTES_GOAL,WEEKLY,150.0,NaN,NEW,False,NaN,NaN,2024-04-23T09:25:09.334+00:00,NaN
2,CALORIES_BURNED_GOAL,DAILY,2600.0,184.0,INPROGRESS_SO_FAR_MISSED,False,2024-08-09,2024-08-10,2016-11-15T15:15:08.000+00:00,NaN
3,STEPS_GOAL,DAILY,10000.0,131.0,INPROGRESS_SO_FAR_MISSED,False,2024-08-09,2024-08-10,2016-11-15T15:15:08.000+00:00,NaN


#### 01.01.04 Biometrics/Glucosa

El archivo CSV de Biometrics de Fitbit contiene datos relacionados con mediciones biométricas personales que se han registrado en tu cuenta. Estos datos incluyen métricas como peso, porcentaje de grasa corporal, masa muscular, índice de masa corporal (IMC), y otras métricas relacionadas con la composición corporal y la salud general. Estos datos son útiles para monitorear cambios en tu cuerpo a lo largo del tiempo y para evaluar tu progreso hacia objetivos de salud específicos.

Tipos de Datos en el CSV de Biometrics:
- **Date (Fecha):**
	- La fecha en que se registró la medición biométrica. Esto te permite ver cómo han cambiado tus métricas a lo largo del tiempo.
- **Weight (Peso):**
	- Tu peso corporal en la fecha especificada. Este valor puede estar en kilogramos (kg), libras (lbs), o la unidad que hayas configurado en tu cuenta de Fitbit.
- **Body Fat Percentage (Porcentaje de grasa corporal):**
	- El porcentaje de tu peso total que está compuesto por grasa. Es una métrica importante para evaluar la composición corporal.
- **BMI (Body Mass Index, Índice de Masa Corporal):**
	- El índice de masa corporal es una medida que relaciona tu peso con tu altura para categorizar tu estado de salud (bajo peso, peso normal, sobrepeso, etc.).
- **Lean Mass (Masa magra):**
	- La masa magra es el peso de todo tu cuerpo menos la grasa corporal. Incluye músculos, huesos, agua, etc.
- **Muscle Mass (Masa muscular):**
	- La cantidad de masa muscular que tienes, medida en kilogramos o libras. Es una métrica clave para evaluar la fuerza y la salud muscular.
- **Bone Mass (Masa ósea):**
	- La cantidad de masa ósea en tu cuerpo. Este valor generalmente no cambia mucho con el tiempo, pero es importante para evaluar la salud ósea.
- **Hydration (Hidratación):**
	- El porcentaje de agua en tu cuerpo. Mantener un nivel de hidratación adecuado es esencial para la salud general.
- **Resting Heart Rate (Frecuencia cardíaca en reposo):**
	- Tu frecuencia cardíaca promedio mientras estás en reposo. Es un indicador importante de tu condición cardiovascular y tu nivel de forma física.
- **Blood Pressure (Presión arterial):**
	- Tu presión arterial registrada, si se ha medido y sincronizado con Fitbit.
- **Oxygen Saturation (Saturación de oxígeno):**
	- La cantidad de oxígeno en tu sangre, medida en porcentaje. Es importante para evaluar la función respiratoria y la oxigenación general del cuerpo.

In [9]:
path_biometrics = 'Datos/Fitbit/Biometrics'

biometrics = llamar_archivos(path_biometrics)
biometrics.head(4)

,time,value,unit,data_source,measurement_type,medical_codes,tags,no data,min,max,days,enabled
0,2024-02-01T07:43+01:00,101.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN
1,2024-02-02T08:10+01:00,99.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN
2,2024-02-03T08:09+01:00,98.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN
3,2024-02-04T08:37+01:00,98.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN


#### 01.01.05 Blocked Users
- Este csv no se va a utilizar para no mostrar datos de terceros.

#### 01.01.06 Daily Readiness

El archivo “Daily Readiness” de Fitbit contiene datos relacionados con tu nivel de preparación diaria, una métrica que Fitbit calcula para ayudarte a entender cómo está tu cuerpo preparado para enfrentar el día en términos de actividad física y recuperación. Este indicador se basa en varios factores, como el sueño, la actividad reciente y la frecuencia cardíaca en reposo, para ofrecerte una puntuación que te guía sobre si deberías enfocarte en un día de ejercicio intenso o en uno de recuperación.

- Tipos de Datos en el CSV de Daily Readiness:
    - **Date (Fecha):**
	    La fecha en la que se calculó la puntuación de preparación diaria. Esto te permite ver tu puntuación a lo largo del tiempo.
	- **Readiness Score (Puntuación de preparación):**
	    La puntuación total que indica tu nivel de preparación para el día. Suele ser un valor numérico que va de 0 a 100, donde una puntuación más alta sugiere que estás más preparado para actividades físicas intensas.
	- **Sleep Score Contribution (Contribución de la puntuación de sueño):**
	    Esta métrica muestra cómo ha contribuido la calidad y duración de tu sueño a la puntuación de preparación diaria. El sueño adecuado mejora esta parte de la puntuación.
	- **Activity Score Contribution (Contribución de la puntuación de actividad):**
	    Muestra cómo tu actividad reciente (generalmente la cantidad de ejercicio y esfuerzo físico en los días anteriores) ha afectado tu preparación. Un equilibrio entre actividad y descanso mejora esta puntuación.
	- **Heart Rate Variability Contribution (Contribución de la variabilidad de la frecuencia cardíaca, HRV):**
	    La variabilidad de la frecuencia cardíaca es una medida de las variaciones en el tiempo entre cada latido del corazón. Una HRV alta generalmente indica que tu cuerpo está bien recuperado, lo que aumenta tu puntuación de preparación.
	- **Resting Heart Rate Contribution (Contribución de la frecuencia cardíaca en reposo):**
	    Muestra cómo tu frecuencia cardíaca en reposo influye en tu preparación diaria. Una frecuencia cardíaca más baja generalmente sugiere una mejor condición física y mayor preparación.
	- **Fatigue Contribution (Contribución del nivel de fatiga):**
	    Indica cuánto influye la fatiga en tu preparación. Si tienes un nivel alto de fatiga, esto reducirá tu puntuación de preparación.
	- **Daily Activity Contribution (Contribución de la actividad diaria):**
	    Refleja cómo tu actividad física diaria está afectando tu preparación para los días siguientes.


In [10]:
path_daily_readliness = 'Datos/Fitbit/Daily Readiness'

preparacion_diaria = llamar_archivos(path_daily_readliness)
preparacion_diaria.head(4)

,date,readiness_score_value,readiness_state,activity_subcomponent,sleep_subcomponent,hrv_subcomponent,activity_state,sleep_state,hrv_state,property_type,value,last_update
0,2023-12-31,65.0,HIGH,65.0,92.0,54.0,LOW,HIGH,MEDIUM,NaN,NaN,NaN
1,2023-12-30,11.0,LOW,11.0,98.0,43.0,HIGH,HIGH,MEDIUM,NaN,NaN,NaN
2,2023-12-29,60.0,MEDIUM,60.0,99.0,40.0,MEDIUM,HIGH,MEDIUM,NaN,NaN,NaN
3,2023-12-28,80.0,HIGH,80.0,97.0,48.0,LOW,HIGH,MEDIUM,NaN,NaN,NaN


#### 01.01.07 Discover 

El archivo CSV de Discover de Fitbit generalmente contiene datos relacionados con el uso de la función de descubrimiento de contenido en la plataforma Fitbit. Esta función te sugiere y permite explorar diversos tipos de contenido, como ejercicios, meditación, artículos sobre salud, y otros recursos que pueden ayudarte a mejorar tu bienestar general.

Tipos de Datos en el CSV de Discover:
- **Date (Fecha):**
	La fecha en la que interactuaste con el contenido sugerido en la sección de “Discover”. Esto te permite rastrear cuándo accediste a los diferentes tipos de contenido.
- **Content Title (Título del contenido):**
	El título del contenido que exploraste o al que accediste a través de la sección de “Discover”. Puede ser el nombre de un artículo, una sesión de ejercicio, una meditación guiada, etc.
- **Content Type (Tipo de contenido):**
	Especifica el tipo de contenido que exploraste. Puede incluir categorías como “Ejercicio”, “Meditación”, “Nutrición”, “Sueño”, o “Artículos de salud”. Este campo te ayuda a entender qué tipo de recursos estás utilizando más frecuentemente.
- **Category (Categoría):**
	Una clasificación más amplia del contenido, como “Salud y Bienestar”, “Actividad Física”, “Mindfulness”, etc. Esto agrupa el contenido en temas generales y te ayuda a analizar cuáles son tus áreas de interés principales.
- **Interaction Type (Tipo de interacción):**
	Describe cómo interactuaste con el contenido. Podría incluir acciones como “Visto”, “Completado”, “Guardado para después”, etc. Esto te ayuda a entender no solo qué contenido te interesa, sino también cómo lo consumes.
- **Duration (Duración):**
	La duración del contenido o el tiempo que pasaste interactuando con él. Esto es especialmente relevante para sesiones de ejercicio o meditación.
- **Completion Status (Estado de finalización):**
	Indica si completaste el contenido o no. Esto es útil para ver qué actividades o recursos sigues hasta el final.
- **Link to Content (Enlace al contenido):**
	Si el contenido está disponible en línea, este campo puede incluir un enlace directo para acceder al mismo nuevamente.


In [11]:
path_discover = 'Datos/Fitbit/Discover'

discover_df = llamar_archivos(path_discover)
discover_df.head(4)

,date,id,title,bundle_id,content_type,rating
0,2024-08-01,07ffe695da2fb5c59a84a97d52f7c0e5,The reset,mindfulness,audio,5
1,2024-08-01,10e77f9dff52b35b5a4d948f826780b4,Learn the one-minute meditation,mindfulness,audio,5
2,2024-08-01,a5e1340025a3b6854808cd70ddde268c,Mindful moment meditation,mindfulness,audio,5
3,2024-08-01,a654a90946aa34a3023336fe2a7f758c,Calm and rest,mindfulness,audio,5


#### 01.01.08 Fitbit Care or Programs 
- Sin datos

#### 01.01.09 Friends
- Este csv no se va a utilizar para no mostrar datos de terceros.

#### 01.01.10 Premium

- Mi enfoque principal es analizar los datos de salud y actividad física recopilada mientras era usuario premium. No se necesita incluir user_premiumSubscription y Transaction. Los datos adicionales proporcionados por la suscripción premium (como métricas avanzadas de sueño, readiness score, etc.) aportan datos que se se encontrarán en los CSVs correspondientes a esas métricas, como “Daily Readiness”, “Advanced Sleep”, “Active Zone Minutes”, etc.

#### 01.01.11 Global Export Data

- En la exportación global de datos de Fitbit, los archivos JSON suelen contener una gran variedad de datos estructurados que se dividen en diferentes categorías relacionadas con la actividad física, salud, sueño, y otros aspectos del bienestar que Fitbit monitorea. Los archivos son parte de esta exportación global y pueden incluir datos detallados sobre varios aspectos del uso de Fitbit.

- Resumen general de los datos de estos archivos JSON:
- **Activity Data (Datos de actividad física):**
    - **Pasos:**
    Datos sobre la cantidad de pasos que has dado en diferentes intervalos de tiempo (minutos, horas, días).
	- **Calorías Quemadas:**
    Información sobre las calorías quemadas durante distintas actividades.
	- **Distancia:**
    Registro de la distancia recorrida en diferentes períodos.
	- **Minutos Activos:**
    Datos sobre el tiempo que pasaste haciendo actividad física moderada a intensa.
	- **Elevación:**
    Pisos subidos y metros o pies ascendidos.
	- **Zonas de Frecuencia Cardíaca:**
    Tiempo pasado en diferentes zonas de frecuencia cardíaca durante el ejercicio.
- **Sleep Data (Datos de sueño):**
    - **Duración del sueño:**
    Tiempo total de sueño por noche.
	- **Fases del sueño:**
    Detalles sobre el tiempo pasado en diferentes fases del sueño (ligero, profundo, REM).
	- **Interrupciones:**
    Cantidad y duración de las veces que te despertaste durante la noche.
- **Heart Rate Data (Datos de frecuencia cardíaca):**
    - **Frecuencia cardíaca en reposo:**
    Tu frecuencia cardíaca promedio mientras estás en reposo.
	- **Frecuencia cardíaca durante el ejercicio:**
    Detalles sobre tu frecuencia cardíaca durante actividades físicas específicas.
	- **Variabilidad de la frecuencia cardíaca (HRV):**
     Datos sobre la variabilidad en el tiempo entre latidos, lo que puede indicar tu nivel de recuperación y estrés.
- **Nutrition Data (Datos de nutrición):**
    - **Ingesta de alimentos:**
    Información sobre las calorías y macronutrientes que has consumido, si has registrado tu ingesta de alimentos.
	- **Hidratación:**
    Datos sobre la cantidad de agua que has consumido.
- **Device Information (Información del dispositivo):**
    - **Batería:**
    Nivel de batería de tu dispositivo Fitbit.
	- **Sincronización:**
    Registro de cuándo y cómo se ha sincronizado tu dispositivo con la app de Fitbit.
- **Stress and Mindfulness (Datos de estrés y mindfulness):**
    - **Sesiones de mindfulness:**
    Detalles sobre las sesiones de respiración guiada o meditación que has realizado.
	- **Nivel de estrés:**
    Si tu dispositivo lo permite, puede incluir información sobre tu nivel de estrés medido a través de la variabilidad de la frecuencia cardíaca o sensaciones registradas.
- **Menstrual Health Tracking (Seguimiento de salud menstrual):**
    - **Ciclo menstrual:**
    Información sobre el seguimiento de tu ciclo menstrual, si lo has registrado.
	- **Síntomas:**
    Registro de síntomas relacionados con tu ciclo.
- **Notifications (Notificaciones):**
    - **Notificaciones de salud:**
    Datos sobre las alertas y notificaciones que has recibido relacionadas con tu salud y actividad.
- **User Profile Data (Datos de perfil de usuario):**
    - **Información personal:**
    Datos básicos como edad, sexo, altura, peso (si lo has registrado en la app).


In [12]:
path_global_export_data = 'Datos/Fitbit/Global Export Data'

global_data_df = llamar_archivos(path_global_export_data, 'altitude')
global_data_df.head(4)

,dateTime,value
0,2023-01-12 11:11:00,10
1,2023-01-12 11:43:00,20
2,2023-01-12 13:51:00,10
3,2023-01-12 14:07:00,10


### 02.01 EDA Apple

### 03.01 EDA datos manuales

In [13]:
pesos_manual = pd.read_csv('Datos/Manual/pesos_manuales.csv')

# La columna desnuda está en 1 para si y en 0 para vestida
pesos_manual.head(4)

,fecha,peso,desnuda
0,8/12/2023,119.9,0
1,5/12/2023,121.4,0
2,9/12/2023,120.2,0
3,16/12/2023,119.9,0


In [14]:
col_peso_manual = {
    'fecha': 'Fecha',
    'peso': 'Peso',
    'desnuda': 'Domicilio'
}

In [15]:
pesos_manual.rename(columns = col_peso_manual, inplace = True)
pesos_manual.sort_values(by = 'Fecha', ascending = True)
pesos_manual.head(4)

,Fecha,Peso,Domicilio
0,8/12/2023,119.9,0
1,5/12/2023,121.4,0
2,9/12/2023,120.2,0
3,16/12/2023,119.9,0


In [16]:
pesos_manual.reset_index(drop = True, inplace = True)
pesos_manual.head(4)

,Fecha,Peso,Domicilio
0,8/12/2023,119.9,0
1,5/12/2023,121.4,0
2,9/12/2023,120.2,0
3,16/12/2023,119.9,0


In [17]:
# Agrupar por 'Fecha' y calcular la media de 'Peso'
pesos_manual_media_diaria = pesos_manual.groupby('Fecha', as_index = False).mean()

pesos_manual_media_diaria.head(4)


,Fecha,Peso,Domicilio
0,04/02/2024,113.6,1.0
1,06/4/2025,106.7,0.0
2,1/4/2024,106.9,0.0
3,10/01/2024,116.4,0.5


In [18]:
pesos_manual_media_diaria.drop('Domicilio', axis = 1, inplace = True)
pesos_manual_media_diaria.tail(4)

,Fecha,Peso
38,8/06/2024,103.5
39,8/12/2023,119.9
40,9/08/2024,102.3
41,9/12/2023,120.2


In [19]:
pesos_manual_ok = pesos_manual_media_diaria.copy()
pesos_manual_ok.sample(4)

,Fecha,Peso
34,27/07/2024,104.45
8,16/03/2024,110.20
25,22/01/2024,115.20
33,27/01/2024,115.60


### 01.04 Concatenación Dataframes para el dataframe final con todos los datos a utilizar para nuestra predicción
    - pesos_manual_ok
    -